<a href="https://colab.research.google.com/github/taravatp/Panopic-Feature-Pyramid-Network/blob/main/training/train_panoptic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# installing and importing required packages

In [ ]:
cd /content/drive/MyDrive/panoptic_FPN

/content/drive/MyDrive/panoptic_FPN


In [ ]:
!pip install import_ipynb 

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=76b71b192535090d8f96ab56f6102f680d67408e39eb5b59b1a6a9b412f23c7e
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import import_ipynb
from coco_panoptic_dataset import panoptic_dataset
from models.panoptic_model import PanopticSegmentationModel

import matplotlib.pyplot as plt
import time
import numpy as np

import utils
import transforms as T

importing Jupyter notebook from coco_panoptic_dataset.ipynb
     |████████████████████████████████| 41 kB 434 kB/s 
     |████████████████████████████████| 217 kB 15.7 MB/s 
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 10.9 MB 3.5 MB/s 
     |████████████████████████████████| 58 kB 4.5 MB/s 
     |████████████████████████████████| 51 kB 6.4 MB/s 
importing Jupyter notebook from /content/drive/MyDrive/panoptic_FPN/models/panoptic_model.ipynb
device: cpu


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


# Instantiating the Dataset/Model/Optimizer/HyperParameters

In [ ]:
batch_size = 2
dataset_train = panoptic_dataset(flag='train')
dataset_test = panoptic_dataset(flag='test')

data_loader_train = torch.utils.data.DataLoader(dataset_train,batch_size=batch_size,shuffle=True,collate_fn=utils.collate_fn,num_workers=2)
data_loader_test = torch.utils.data.DataLoader(dataset_test,batch_size=batch_size,shuffle=True,collate_fn=utils.collate_fn,num_workers=2) #az collate_FN estefade mikonim chon niaz darim size e tamame vurudiha baham barabar bashand

loading annotations into memory...
Done (t=3.86s)
creating index...
index created!
loading annotations into memory...
Done (t=2.10s)
creating index...
index created!
loading annotations into memory...
Done (t=1.10s)
creating index...
index created!
loading annotations into memory...
Done (t=1.58s)
creating index...
index created!


In [ ]:
model = PanopticSegmentationModel(81,92).to(device)

learning_rate = 1e-4
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=3,gamma=0.1)
loss_function = nn.CrossEntropyLoss()

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

# training

In [ ]:
number_batches = len(data_loader_train)
print(number_batches)

750


In [ ]:
num_epochs = 40

instance_loss_per_epoch = []
semantic_loss_per_epoch = []
total_loss_per_epoch = []

for epoch in range(num_epochs):

  print('starting epoch number:',epoch)
  model.train()

  time_start = time.time()
  loss_accum_instance = 0
  loss_accum_semantic = 0
  loss_accum_all = 0
  loss_accum_classifer = 0
  loss_accum_regression = 0
  loss_accum_mask = 0
  loss_segmentation = 0

  for iter,batch in enumerate(data_loader_train):

    images = batch[0] #tuple
    images = list(image.to(device) for image in images) #list
    images = torch.stack(images).to(device)

    semantic_targets = batch[1]
    semantic_targets = list(semantic_target.to(device).long() for semantic_target in semantic_targets)
    semantic_targets = torch.stack(semantic_targets).to(device).long()

    instance_targets = batch[2]
    instance_targets = [{k:v.to(device) for k,v in t.items()} for t in instance_targets]

    loss_dict_instance,predicted_semantic_masks = model(images,instance_targets) #feed forwarding

    #calculating loss values
    loss_instance = sum(loss for loss in loss_dict_instance.values())
    print(predicted_semantic_masks.shape)
    print(semantic_targets.shape)
    loss_semantic = loss_function(predicted_semantic_masks,semantic_targets)
    total_loss = loss_semantic*0.25 + loss_instance*0.75

    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    loss_accum_instance += loss_instance.item()
    loss_accum_semantic += loss_semantic.item()
    loss_accum_all += total_loss.item()
    loss_classifier = loss_dict_instance['loss_classifier'].item()
    loss_box_reg = loss_dict_instance['loss_box_reg'].item()
    loss_mask = loss_dict_instance['loss_mask'].item()
    loss_accum_classifer += loss_classifier
    loss_accum_regression += loss_box_reg
    loss_accum_mask += loss_mask
    
    if iter%100 == 0 :
      print(f"epoch:{epoch} *** iter:{iter} *** loss_semantic:{loss_semantic} *** loss_instance:{loss_instance} *** loss_classification:{loss_classifier} *** loss_regression:{loss_box_reg} *** loss_mask:{loss_mask}")

  
  lr_scheduler.step()
  instance_loss_per_epoch.append(loss_accum_instance/len(data_loader_train))
  semantic_loss_per_epoch.append(loss_accum_semantic/len(data_loader_train))
  total_loss_per_epoch.append(loss_accum_all/len(data_loader_train))

  print(f"end of epoch number {epoch}")
  print(f"time elapsed: {time.time() - time_start}")
  print(f"train_instance_loss: {loss_accum_instance/number_batches}")
  print(f"train_semantic_loss: {loss_accum_semantic/number_batches}")
  print(f"classification_loss: {loss_accum_classifer/number_batches}")
  print(f"regression_loss: {loss_accum_regression/number_batches}")
  print(f"mask_loss: {loss_accum_mask/number_batches}")

  torch.save(model.state_dict(), f"panoptic_Model{epoch}_batch_{batch_size}.pth")
